In [1]:
#!pip install tabula-py

In [2]:
import tabula
import pandas as pd
import itertools

In [3]:
def get_column_list(dataframe_obj):
    '''
    Puts all the column names from a list of dataframes into a list
    
    input: list of dataframes
    output: lists of column names from all the dataframes
    '''
    columns = [list(dataframe_obj[i].columns) for i in range(len(dataframe_obj))]
    return columns

def get_series_list(dataframe_obj, columns_list):
    '''
    Turns all the columns in each of the dataframes to series and puts them all in a list
    
    input: list of dataframes and the column list from get_column_list
    output: list of panda series objects
    '''
    series_list = [dataframe_obj[i] for i in columns_list]
    series_list2 = [series_list[i].dropna() for i in range(len(series_list))]
    return series_list2

def append_columns(filtered_columns, to_series):
    '''
    Appends items to a pandas series object
    
    input: items to append and an object to append them to
    output: a series object with the appended items
    '''
    to_append = pd.Series(filtered_columns) 
    return to_series.append(to_append, ignore_index=True)

##### Will output a list of dataframes (1 for each table in the pdf) BUT
1) The table has lost nearly all its organization

2) Some of the column names should actually be row values

3) NaN values litter the output

In [ ]:
df = tabula.read_pdf(r"C:\Users\rayde\Downloads\2020-2021-Approved-Stock-List-20200926.pdf", pages = 'all', lattice = False, multiple_tables=True)
df

In [5]:
#Get the list of columns in each of the dataframes 
columns_list = get_column_list(df)

In [ ]:
columns_list

#### Because Tabula outputs messy data for this pdf, we will need to: 
    1. Turn each dataframe into a list of pandas series objects to remove the columns (We only want one column)
    2. Use pd.concat to rebuild the series objects into pandas objects with only one column

##### The cell below turns each dataframe into a list of series.

In [ ]:
series_list = [get_series_list(df[x], columns_list[x]) for x in range(len(df))]
series_list

3. Run pd.concat on each list of series and then again on the output
4. Once we run pd.concat for the second time, we will have a dataframe with only one column 

##### The cell below turns our list of list of pandas series objects into one pandas object with one column

In [8]:
data = list(map(lambda x: pd.concat(series_list[x], ignore_index=True), range(len(series_list))))
data = pd.concat(data)
data

0                      Co
1     Acorn International
2                     Inc
3          Amazon.com Inc
4          American Eagle
             ...         
30             Utilities:
31     Telecommunications
32             Utilities:
33     Telecommunications
34             Utilities:
Length: 2534, dtype: object

###### Two filters
3. Filter the series by length<7 because the tickers all have less than seven characters (Some of the tickers are quite long)
4. Filter by words we don't want that are less than seven characters in length.

##### The cell below filters data by character length and words less than 7 characters that we don't want in our list

In [9]:
bad_words = ["Retail", "Media", "Foods", "Nasdaq", "EQUITY", "Cement", "Copper", "ENERGY", "Inc", "Co", "STYLE", "TICKER", "Balcão"]
filtered_data = data[(data.str.len()<7) & (data.isin(bad_words)==False)]
filtered_data

32     ATV
33    AMZN
34     AEO
35    APTV
36     AVP
      ... 
2      PLC
8      UU.
9       VZ
10     VOD
11     VOD
Length: 406, dtype: object

##### Tabula took some of the tickers from the PDF file and assigned them to column names. 
We want to make sure we don't leave any of the tickers out. So, we also need to: 
5. Filter the column list by bad words and length less than 7 
6. Append to the filtered_data pandas series object.

##### The cell below uses itertools to unzip the list of lists of column names into one list and then filters it.

In [10]:
'''
Unzips the column list and puts all the column names into one list (versus having a list of lists)
'''

columns_one_list = list(itertools.chain.from_iterable(columns_list)) #itertools will turn the list of list into one list
columns_list_filtered = [x for x in columns_one_list if (len(x) < 7) & (x not in bad_words)] #Filter by less than 7 and bad words


In [11]:
#append the columns to the filtered_data_series
final_lists = append_columns(columns_list_filtered, filtered_data)

In [12]:
final_lists

0        ATV
1       AMZN
2        AEO
3       APTV
4        AVP
       ...  
423    RAIL3
424     ADBE
425     ANET
426      AEP
427    EGIE3
Length: 428, dtype: object

##### There are duplicates in our pandas series
7. Remove duplicate values

In [13]:
final_lists[final_lists=='VOD']

404    VOD
405    VOD
dtype: object

In [14]:
#We need to drop duplicates
final_lists = final_lists.drop_duplicates()

In [15]:
final_lists

0        ATV
1       AMZN
2        AEO
3       APTV
4        AVP
       ...  
423    RAIL3
424     ADBE
425     ANET
426      AEP
427    EGIE3
Length: 346, dtype: object

In [16]:
final_lists.to_excel('stocklist.xlsx')